# 2024 Bike Availability Prediction

In [ ]:
# Importar librerías que vamos a estar usando en general
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load data

Descarga de datos en el caso de Windows

In [ ]:
# Librerías para poder descargar los datos
! pip install py7zr
import os
import requests
import py7zr

# Lista de tuplas de meses
i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))

# Directorio donde guardar los archivos descargados
download_dir = "C:/Users/34627/OneDrive/Postgrado UB/PROYECTO FINAL"

# Iterar sobre años y meses
for year in [2023, 2022, 2021, 2020]:
    for month, month_name in i2m:        
        # URL del archivo a descargar
        url = f"https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"
        
        # Nombre del archivo
        filename = os.path.join(download_dir, f"{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z")
        
        # Descargar el archivo
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)
        
        # Descomprimir el archivo
        with py7zr.SevenZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(download_dir)
        
        # Eliminar el archivo comprimido
        os.remove(filename)


In [ ]:
## Cargamos los datos

# Vamos a predeterminar los datos a ordenar en el dataset
years = ['2020', '2021', '2022', '2023']
months = ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']
indx = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

dir = "C:/Users/34627/OneDrive/Postgrado UB/PROYECTO FINAL/raw_data"
data = pd.DataFrame()

# Recorreremos los años, índices y meses
for year in years:
    for index, month in zip(indx, months):
        # Nombre del archivo
        nombre_archivo = '{}_{:02d}_{}_BicingNou_ESTACIONS.csv'.format(year, index, month)
        # Verificación de archivo
        if nombre_archivo in os.listdir(dir):
            # Leer el archivo CSV y cargar los datos en el DataFrame
            month_year = pd.read_csv(os.path.join(dir, nombre_archivo))
            # Concatenar los datos al DataFrame 
            data = pd.concat([data, month_year], ignore_index=True)

# Visualizar la tabla de datos completa
print(data.head())


## Data cleaning

In [ ]:
# Eliminamos columnas no relevantes (V1, traffic = todo son Nans)
columnas_no = ['traffic']
data = data.drop(columnas_no, axis = 1)

In [ ]:
# Detectar filas con valores NaN
filas_con_nans = data[data.isna().any(axis=1)]

# Visualizar las filas con valores NaN
print("Filas con valores NaN:")
print(filas_con_nans)

In [20]:
# Reemplazar NaN con la media
data.fillna(data.mean(), inplace=True)

In [ ]:
# Normalización de los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[['caracteristica1', 'caracteristica2']] = scaler.fit_transform(data[['caracteristica1', 'caracteristica2']])